<a href="https://colab.research.google.com/github/GonzaloHernandez10/online-retail-II-data-quality-framework/blob/main/Proyecto2_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PREPARACIÓN DEL AMBIENTE DE TRABAJO Y CARGA DE DATOS

In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
import os
folderpath = '/content/drive/My Drive/Proyectos - Analisis de Datos/Proyectos - Curriculares/Proyecto 2 - Análisis de ventas - Online Retail II'
os.chdir(folderpath)
!ls

 online_retail_II.csv  'Proyecto2 - Script.ipynb'   README.gdoc


In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [64]:
filename = 'online_retail_II.csv'
df_online_retail = pd.read_csv(filename, encoding='latin1')
df_online_retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom


# DIAGNOSTICO CLÍNICO INICIAL (DATA PROFILING)

In [65]:
#####################################
# Inspección del shape del dataframe
#####################################
df_online_retail.shape

(541910, 8)

In [66]:
###################################
# Inspección general del dataframe
###################################
df_online_retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      541910 non-null  object 
 1   StockCode    541910 non-null  object 
 2   Description  540456 non-null  object 
 3   Quantity     541910 non-null  int64  
 4   InvoiceDate  541910 non-null  object 
 5   Price        541910 non-null  float64
 6   Customer ID  406830 non-null  float64
 7   Country      541910 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [67]:
#################################################
# Cuantificación del volumen inicial de registros
##################################################
total_registros_brutos = len(df_online_retail.index)
print(f'El total de registros brutos del DataFrame es {total_registros_brutos}')

El total de registros brutos del DataFrame es 541910


In [68]:
######################################
# Verificación de los campos críticos
######################################
def check_campos_criticos(df, campos_criticos):
  for campo in campos_criticos:
    if campo in df.columns:
      print(f'✔️El campo crítico {campo} existe en el dataframe')

campos_criticos = ['Description', 'Quantity', 'Price']
check_campos_criticos(df_online_retail, campos_criticos)

✔️El campo crítico Description existe en el dataframe
✔️El campo crítico Quantity existe en el dataframe
✔️El campo crítico Price existe en el dataframe


In [69]:
##############################################
# Inspección de valores nulos en el dataframe
##############################################
df_online_retail.isnull().sum()

,0
Invoice,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
Price,0
Customer ID,135080
Country,0


In [70]:
########################################
# Identificación del % de valores nulos
# ((valores_nulos / total_registros_brutos) * 100)
########################################
valores_nulos = df_online_retail.isnull().sum()
porcentaje_valores_nulos = (valores_nulos / total_registros_brutos) * 100

resumen_nulos = pd.DataFrame({
  'Valores nulos': valores_nulos,
  'Porcentaje de valores nulos': porcentaje_valores_nulos
})

print(f'El resumen de valores nulos es el siguiente:\n\n{resumen_nulos[resumen_nulos['Valores nulos'] > 0]}')

El resumen de valores nulos es el siguiente:

             Valores nulos  Porcentaje de valores nulos
Description           1454                     0.268310
Customer ID         135080                    24.926648


In [71]:
###################################################################
# Cálculo de Net Sales con los valores brutos de Quantity y Prince
# Net Sales = (Quantity * Price)
###################################################################
df_online_retail['Net Sales'] = df_online_retail['Quantity'] * df_online_retail['Price']
ventas_netas_brutas = df_online_retail['Net Sales'].sum()
print(f'Las ventas netas brutas sin tratamiento en Quantity y Price es: ${ventas_netas_brutas:,.2f}')

Las ventas netas brutas sin tratamiento en Quantity y Price es: $9,747,765.93


In [72]:
############################################################################
# Verificación de valores duplicados por campos criticos y el campo Invoice
############################################################################
valores_duplicados_criticos = df_online_retail.duplicated(subset=['Invoice', 'Description', 'Quantity', 'Price'])
print(f'El total de valores duplicados por campos criticos es: {valores_duplicados_criticos.sum()}')

El total de valores duplicados por campos criticos es: 5339


In [73]:
#####################################
# Verificación de valores duplicados
#####################################
valores_duplicados_general = df_online_retail.duplicated().sum()
print(f'El total de valores duplicados es: {valores_duplicados_general}')

El total de valores duplicados es: 5268


# DIAGNOSTICO DE LAS VALIDACIONES DE NEGOCIO (DATA PROFILING)

In [74]:
############################
# Validación de completitud
############################
campos_criticos = ['Invoice', 'Description', 'Quantity', 'Price']
validacion_integridad = df_online_retail[df_online_retail[campos_criticos].isna().any(axis=1)]
validacion_integridad.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1454 entries, 622 to 538554
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Invoice      1454 non-null   object 
 1   StockCode    1454 non-null   object 
 2   Description  0 non-null      object 
 3   Quantity     1454 non-null   int64  
 4   InvoiceDate  1454 non-null   object 
 5   Price        1454 non-null   float64
 6   Customer ID  0 non-null      float64
 7   Country      1454 non-null   object 
 8   Net Sales    1454 non-null   float64
dtypes: float64(3), int64(1), object(5)
memory usage: 113.6+ KB


In [75]:
#######################################
# Validación de integridad de ingresos
#######################################
mask = (df_online_retail['Quantity'] < 0) | (df_online_retail['Price'] < 0)
validacion_integridad = df_online_retail[mask]
validacion_integridad.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10626 entries, 141 to 541717
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Invoice      10626 non-null  object 
 1   StockCode    10626 non-null  object 
 2   Description  9764 non-null   object 
 3   Quantity     10626 non-null  int64  
 4   InvoiceDate  10626 non-null  object 
 5   Price        10626 non-null  float64
 6   Customer ID  8905 non-null   float64
 7   Country      10626 non-null  object 
 8   Net Sales    10626 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 830.2+ KB


In [76]:
####################################################
# Validación de integridad de mitigación de riesgos
####################################################
mask = (df_online_retail['Price'] < 0)
validacion_mitigacion_riesgo = df_online_retail[mask]
validacion_mitigacion_riesgo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 299983 to 299984
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Invoice      2 non-null      object 
 1   StockCode    2 non-null      object 
 2   Description  2 non-null      object 
 3   Quantity     2 non-null      int64  
 4   InvoiceDate  2 non-null      object 
 5   Price        2 non-null      float64
 6   Customer ID  0 non-null      float64
 7   Country      2 non-null      object 
 8   Net Sales    2 non-null      float64
dtypes: float64(3), int64(1), object(5)
memory usage: 160.0+ bytes


In [77]:
#############################
# Validación de trazabilidad
#############################
mask = df_online_retail['Invoice'].str.startswith('C')
validacion_trazabilidad = df_online_retail[mask]
validacion_trazabilidad.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9288 entries, 141 to 541717
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Invoice      9288 non-null   object 
 1   StockCode    9288 non-null   object 
 2   Description  9288 non-null   object 
 3   Quantity     9288 non-null   int64  
 4   InvoiceDate  9288 non-null   object 
 5   Price        9288 non-null   float64
 6   Customer ID  8905 non-null   float64
 7   Country      9288 non-null   object 
 8   Net Sales    9288 non-null   float64
dtypes: float64(3), int64(1), object(5)
memory usage: 725.6+ KB


In [78]:
#########################
# Validación de unicidad
#########################
mask = df_online_retail.duplicated(subset=['Invoice', 'Description', 'Quantity', 'Price'])
validacion_unicidad = df_online_retail[mask]
validacion_unicidad.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5339 entries, 517 to 541701
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Invoice      5339 non-null   object 
 1   StockCode    5339 non-null   object 
 2   Description  5339 non-null   object 
 3   Quantity     5339 non-null   int64  
 4   InvoiceDate  5339 non-null   object 
 5   Price        5339 non-null   float64
 6   Customer ID  5262 non-null   float64
 7   Country      5339 non-null   object 
 8   Net Sales    5339 non-null   float64
dtypes: float64(3), int64(1), object(5)
memory usage: 417.1+ KB


# CATEGORIZACIÓN Y ETIQUETADO (DATA LABELING & CATEGORIZATION)

In [79]:
#################################################
# Creación de las columnas Code Error y Category
#################################################
df_online_retail['Code Error'] = 'N/A'
df_online_retail['Category'] = 'N/A'

In [80]:
#####################################################################
# Identificación de errores por cancelación (BAJA PRIORIDAD) y
# por ajustes del sistema (ALTA PRIORIDAD)
#####################################################################
mask_cancelacion = df_online_retail['Invoice'].str.startswith('C')
mask_ajuste_sistema = (df_online_retail['Quantity'] < 0) & (~mask_cancelacion)

df_online_retail.loc[mask_cancelacion, 'Code Error'] = 'COMMERCIAL_CANCELATION'
df_online_retail.loc[mask_ajuste_sistema, 'Code Error'] = 'SYSTEM_ADJUSTMENT'

In [81]:
#########################################################################
# Identificación de errores para la mitigación de riesgos en los precios
# (BAJA PRIORIDAD)
#########################################################################
mask = df_online_retail['Price'] < 0
df_online_retail.loc[mask, 'Code Error'] = 'INVALID_UNIT_PRICE'

In [82]:
######################################################
# Identificación de errores críticos (ALTA PRIORIDAD)
######################################################
mask = df_online_retail[['Invoice', 'Description', 'Quantity', 'Price']].isna().any(axis=1)
df_online_retail.loc[mask, 'Code Error'] = 'CRITICAL_ERROR'

In [83]:
# ###########################################################
# Categorización de los registros con fallo crítico
###########################################################
mask = (df_online_retail['Code Error'] == 'CRITICAL_ERROR') | (df_online_retail['Code Error'] == 'SYSTEM_ADJUSTMENT')
df_online_retail.loc[mask, 'Category'] = 'INVALID'

In [84]:
###########################################################
# Categorización de los registros sin ningun tipo de fallo
###########################################################
mask = (df_online_retail['Code Error'] == 'N/A') | (df_online_retail['Code Error'] == 'COMMERCIAL_CANCELATION') | (df_online_retail['Code Error'] == 'INVALID_UNIT_PRICE')
df_online_retail.loc[mask, 'Category'] = 'VALID'

In [85]:
###################################################################
# Inspección del resultado del DLA
# Los registros con Code Error = N/A se consideran "Datos Limpios"
###################################################################
conteo_errores = df_online_retail.value_counts('Code Error')
print(f'Conteo de errores:\n\n{conteo_errores}')

conteo_categorias = df_online_retail.value_counts('Category')
print(f'\n\nConteo de categorías:\n\n{conteo_categorias}')

Conteo de errores:

Code Error
N/A                       530692
COMMERCIAL_CANCELATION      9288
CRITICAL_ERROR              1454
SYSTEM_ADJUSTMENT            474
INVALID_UNIT_PRICE             2
Name: count, dtype: int64


Conteo de categorías:

Category
VALID      539982
INVALID      1928
Name: count, dtype: int64


# LIMPIEZA E IMPUTACIÓN DE DATOS (DATA CLEANING & IMPUTATION)

In [98]:
######################################################################
# Eliminación de valores duplicados por campos críticos y Customer ID
######################################################################
df_online_retail = df_online_retail.drop_duplicates(subset=['Customer ID', 'Invoice', 'Description', 'Quantity', 'Price'])
df_online_retail.info()

<class 'pandas.core.frame.DataFrame'>
Index: 536571 entries, 0 to 541909
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      536571 non-null  object 
 1   StockCode    536571 non-null  object 
 2   Description  535117 non-null  object 
 3   Quantity     536571 non-null  int64  
 4   InvoiceDate  536571 non-null  object 
 5   Price        536571 non-null  float64
 6   Customer ID  401568 non-null  float64
 7   Country      536571 non-null  object 
 8   Net Sales    536571 non-null  float64
 9   Code Error   536571 non-null  object 
 10  Category     536571 non-null  object 
dtypes: float64(3), int64(1), object(7)
memory usage: 49.1+ MB


In [99]:
#############################################
# Separación de valores en VALID = DF_VALIDO
# DF_VALIDO
#############################################
df_valido = df_online_retail[df_online_retail['Category'] == 'VALID'].copy()
df_valido.info()

<class 'pandas.core.frame.DataFrame'>
Index: 534643 entries, 0 to 541909
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      534643 non-null  object 
 1   StockCode    534643 non-null  object 
 2   Description  534643 non-null  object 
 3   Quantity     534643 non-null  int64  
 4   InvoiceDate  534643 non-null  object 
 5   Price        534643 non-null  float64
 6   Customer ID  401568 non-null  float64
 7   Country      534643 non-null  object 
 8   Net Sales    534643 non-null  float64
 9   Code Error   534643 non-null  object 
 10  Category     534643 non-null  object 
dtypes: float64(3), int64(1), object(7)
memory usage: 48.9+ MB


In [100]:
#################################################
# Separación de valores en INVALID = DF_INVALIDO
# DF_INVALIDO
#################################################
df_invalido = df_online_retail[df_online_retail['Category'] == 'INVALID'].copy()
df_invalido['Code Error'].value_counts()

,count
Code Error,
CRITICAL_ERROR,1454
SYSTEM_ADJUSTMENT,474


In [101]:
##############################
# Separación de cancelaciones
##############################
df_cancelaciones = df_valido[df_valido['Code Error'] == 'COMMERCIAL_CANCELATION'].copy()
df_cancelaciones.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9250 entries, 141 to 541717
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Invoice      9250 non-null   object 
 1   StockCode    9250 non-null   object 
 2   Description  9250 non-null   object 
 3   Quantity     9250 non-null   int64  
 4   InvoiceDate  9250 non-null   object 
 5   Price        9250 non-null   float64
 6   Customer ID  8871 non-null   float64
 7   Country      9250 non-null   object 
 8   Net Sales    9250 non-null   float64
 9   Code Error   9250 non-null   object 
 10  Category     9250 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 867.2+ KB


In [102]:
###########################################################
# Imputación de Quantity y Price a 0 si (q < 0) or (p < 0)
###########################################################
df_valido['Quantity'] = np.where(df_valido['Quantity'] < 0, 0, df_valido['Quantity'])
df_valido['Price'] = np.where(df_valido['Price'] < 0, 0, df_valido['Price'])

df_valido.info()

<class 'pandas.core.frame.DataFrame'>
Index: 534643 entries, 0 to 541909
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      534643 non-null  object 
 1   StockCode    534643 non-null  object 
 2   Description  534643 non-null  object 
 3   Quantity     534643 non-null  int64  
 4   InvoiceDate  534643 non-null  object 
 5   Price        534643 non-null  float64
 6   Customer ID  401568 non-null  float64
 7   Country      534643 non-null  object 
 8   Net Sales    534643 non-null  float64
 9   Code Error   534643 non-null  object 
 10  Category     534643 non-null  object 
dtypes: float64(3), int64(1), object(7)
memory usage: 48.9+ MB


# OBTENCIÓN DE DATOS PARA EL ANÁLISIS DE CAUSA RAÍZ

In [111]:
#################################################
# ¿Cúal es el porcentaje de errores presentados?
#################################################
conteo_errores_criticos = (len(df_invalido[df_invalido['Code Error'] == 'CRITICAL_ERROR']) / len(df_online_retail)) * 100
print(f'El {conteo_errores_criticos:.2f}% de los registros tienen errores criticos')

conteo_errores_ajuste_sistema = (len(df_invalido[df_invalido['Code Error'] == 'SYSTEM_ADJUSTMENT']) / len(df_online_retail)) * 100
print(f'El {conteo_errores_ajuste_sistema:.2f}% de los registros tienen ajustes de sistema')

El 0.27% de los registros tienen errores criticos
El 0.09% de los registros tienen ajustes de sistema


In [114]:
######################################################
# ¿Cúal es el porcentaje de transacciones canceladas?
######################################################
conteo_errores_cancelacion = (len(df_cancelaciones) / len(df_online_retail)) * 100
print(f'El {conteo_errores_cancelacion:.2f}% de los registros tienen cancelaciones')

El 1.72% de los registros tienen cancelaciones


In [118]:
################################################
# ¿Cúal es el porcentaje de valores duplicados?
################################################
conteo_valores_duplicados = (valores_duplicados_criticos.sum() / total_registros_brutos) * 100
print(f'El {conteo_valores_duplicados:.2f}% de los registros duplicados')

El 0.99% de los registros duplicados


# OBTENCIÓN DE DATOS PARA EL BRIDGE REPORT

In [119]:
####################################################
# Cálculo de las ventas netas con los datos validos
####################################################
df_valido['Net Sales'] = df_valido['Quantity'] * df_valido['Price']
ventas_netas_valido = df_valido['Net Sales'].sum()
print(f'Las ventas netas con datos validos es: ${ventas_netas_valido:,.2f}')

Las ventas netas con datos validos es: $10,641,861.33


In [120]:
#######################################################
# Cálculo de las ventas netas con los datos cancelados
#######################################################
df_cancelaciones['Net Sales'] = df_cancelaciones['Quantity'] * df_cancelaciones['Price']
ventas_netas_cancelaciones = df_cancelaciones['Net Sales'].sum()
print(f'Las ventas netas con datos cancelados es: ${ventas_netas_cancelaciones:,.2f}')

Las ventas netas con datos cancelados es: $-893,971.33


In [126]:
#######################################
# Obtención de ventas netas sanitizada
#######################################
total = ventas_netas_valido + ventas_netas_cancelaciones
print(f'El total de ventas netas es: ${total:,.2f}')

El total de ventas netas es: $9,747,890.00


In [128]:
#############################################################
# Varianza entre total ventas bruta y total venta sanitizada
#############################################################
total_vs = ventas_netas_brutas - total
print(f'Total: {total_vs}')

Total: -124.06999999471009


# EXPORTACIÓN DE LOS DATOS

In [132]:
###################################
# Exportación del DataFrame válido
###################################
df_valido.to_csv('df_valido.csv', index=False)

In [133]:
###################################
# Exportación del DataFrame inválido
###################################
df_invalido.to_csv('df_invalido.csv', index=False)

In [134]:
##############################################
# Exportación del DataFrame con cancelaciones
##############################################
df_cancelaciones.to_csv('df_cancelaciones.csv', index=False)